In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
import os
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def load_images_from_folder(folder,only_path = False, label = ""):
# Load the paths to the images in a directory
# or load the images
    if only_path == False:
        images = []
        for filename in os.listdir(folder):
            img = plt.imread(os.path.join(folder,filename))
            if img is not None:
                images.append(img)
        return images
    else:
        path = []
        for filename in os.listdir(folder):
            img_path = os.path.join(folder,filename)
            if img_path is not None:
                path.append([label,img_path])
        return path

In [30]:
images = []
dirp = "fruit_reco_dataset/train/"
for f in os.listdir(dirp):
    if "jpg" in os.listdir(dirp+f)[0]:
        images += load_images_from_folder(dirp+f,True,label = f)
    else:
        for d in os.listdir(dirp+f):
            images += load_images_from_folder(dirp+f+"/"+d,True,label = f)

# Create a dataframe with the paths and the label for each fruit
df = pd.DataFrame(images, columns = ["fruit", "path"])

l = []
for i in df['path'].values:
    i = i.replace("\\","/")
    l.append(i)

df["path"] = l

print(df.head())


   fruit                                         path
0  Apple   fruit_reco_dataset/train/Apple/apple_1.jpg
1  Apple  fruit_reco_dataset/train/Apple/apple_10.jpg
2  Apple  fruit_reco_dataset/train/Apple/apple_11.jpg
3  Apple  fruit_reco_dataset/train/Apple/apple_12.jpg
4  Apple  fruit_reco_dataset/train/Apple/apple_13.jpg
